In [10]:
import os, glob
import json
import importlib
import radiometric_calib_utils
import mutils
import micasense.imageutils as imageutils
import PIL.Image as Image
import extract_spectral
importlib.reload(radiometric_calib_utils)
importlib.reload(mutils)
importlib.reload(extract_spectral)
import mutils
import extract_spectral as espect
import matplotlib.pyplot as plt
import numpy as np
from math import ceil

c:\Users\PAKHUIYING\Documents\sunglint_correction\extract_spectral.py:220: DeprecationWarning: invalid escape sequence \d
  """


# QAQC of labelled images

We will replot the bboxes on the images again using the info from `saved_bboxes`. The aim is to check if the drawn bboxes is drawn correctly on the images due to some bugs before the program was stable.

Previous bugs:

1. `saved_plots` were not saved together with `saved_bboxes` text files, which makes it hard to validate the bboxes drawn $\rightarrow$ this has been subsequently corrected by saving both the bboxes and plots. 

    Consequence: Some bboxes text files do not have a corresponding saved plots

2. Some bboxes from previous images were transferred over to subsequent images because the bboxes were not cleared ("reset") $\rightarrow$ this has been corrected

    Consequence: Some images had unwanted drawn bboxes which were wrong

3. Normalisation of RGB image $\rightarrow$ this has been subsequently corrected by providing true rgb images

    Consequence: Normalisation may have exaggerated the brightness of the image, resulting in possibly mislabelling regions as glint even though sun glint was not present



If data was transferred from another platform, the file path of the images saved in the text file will not be consistent with the file path in your local machine. We will need to reassign a new directory to the file path in the text file:

For example, your raw images was stored in the C drive, but mine is stored in the D drive, so when the program tries to search for images in my C drive, there is an error because my C drive doesn't have the raw images. This is why we need to reassign the main directory.

- directory (*this is the directory that needs to be reassigned*)
    - survey folder
        - flight folder
            - RawImg folder
                - ...raw images...

To reassign the images to new directory, simply specify where all the survey folders are stored in YOUR local machine e.g. `new_dir = r"D:\EPMC_flight"`

In [12]:
# mutils.get_bboxes_complement("saved_bboxes_1",r"D:\QAQC\QAQC_plots","saved_bboxes_1_continue")

75


['13thSur22Sep_F2_RawImg_IMG_0091_1',
 '13thSur22Sep_F2_RawImg_IMG_0201_1',
 '8thSur11Aug_F4_RawImg_IMG_0280_1',
 '13thSur22Sep_F2_RawImg_IMG_0236_1',
 '1stSur6Apr_F3_RawImg_IMG_0329_1',
 '12thSur13Sep_F1_RawImg_IMG_0000_1',
 '9thSur16Aug_F2_RawImg_IMG_0000_1',
 '13thSur22Sep_F2_RawImg_IMG_0145_1',
 '1stSur6Apr_F2_RawImg_IMG_0000_1',
 '8thSur11Aug_F3_RawImg_IMG_0763_1',
 '1stSur6Apr_F3_RawImg_IMG_0270_1',
 '8thSur11Aug_F2_RawImg_IMG_0062_1',
 '13thSur22Sep_F2_RawImg_IMG_0018_1',
 '8thSur11Aug_F1_RawImg_IMG_0689_1',
 '8thSur11Aug_F1_RawImg_IMG_0424_1',
 '8thSur11Aug_F4_RawImg_IMG_0441_1',
 '1stSur6Apr_F1_RawImg_IMG_0402_1',
 '10thSur24Aug_F1_RawImg_IMG_0016_1',
 '1stSur6Apr_F3_RawImg_IMG_0432_1',
 '8thSur11Aug_F4_RawImg_IMG_0162_1',
 '8thSur11Aug_F3_RawImg_IMG_0540_1',
 '8thSur11Aug_F4_RawImg_IMG_0486_1',
 '8thSur11Aug_F4_RawImg_IMG_0020_1',
 '9thSur16Aug_F1_RawImg_IMG_0000_1',
 '8thSur11Aug_F3_RawImg_IMG_0000_1',
 '1stSur6Apr_F2_RawImg_IMG_0443_1',
 '13thSur22Sep_F2_RawImg_IMG_0425_1',

# Step 1

- combine all `saved_bboxes` together

In [13]:
new_dir = r"D:\EPMC_flight"
ES = espect.VerifyBboxes(r"saved_bboxes_1_continue", 
                         assign_new_dir=new_dir,split_iter=3)
store_bboxes = ES.store_bboxes()
if os.path.exists(list(store_bboxes)[0]):
    ES.plot_bboxes(show_n = None,figsize=(10,20),save=True, plot_dir = r"D:\QAQC")
else:
    print(f"QAQC plots not saved because {list(store_bboxes)[0]} does not exist!\nCheck if {new_dir} is the folder where survey images are stored")

Flights: 100%|██████████| 13/13 [00:00<?, ?it/s]


Else, if no reassignment is needed, just run the following:

In [60]:
ES = espect.VerifyBboxes(r"saved_bboxes")
store_bboxes = ES.store_bboxes()
if os.path.exists(list(store_bboxes)[0]):
    ES.plot_bboxes(show_n = None,figsize=(10,20),save=True)
else:
    print(f"QAQC plots not saved because {list(store_bboxes)[0]} does not exist!")

(919, 1239, 3)
(919, 1239, 3)


Clipping input data to the valid range for imshow with RGB data ([0..1] for floats or [0..255] for integers).


(919, 1239, 3)


# Step 2

Now after the code above has finished running (probably will take very long if you are going through many images), we will start with the QAQC. 

- Make sure that the folder *QAQC_plots* have been created.
- Run `get_training_data.py` (it is the same GUI program that can be used for both labelling, QAQC, and for relabelling! Yay to not creating a separate GUI program!)
- During the text prompt `Enter directory:`, enter the filepath for *QAQC_plots*
- The program works pretty much the same as before, except for the textbox to submit the inputs. In the textbox you can enter the filename e.g. *1stSur6Apr_F1_RawImg_IMG_0005_1* to jump to whatever image you wish to.
![QAQC](example_images/gui_qaqc.png)

- if the bboxes have all been drawn correctly for the current image, click *Next* to skip the current image
- IMPORTANT: if **at least one** of the bboxes are drawn wrongly, you will need to redraw __all__ the bboxes, the previous bboxes are there as guidance if some of the bboxes are correctly drawn (So you can draw over the previous bboxes).

When you have come to the end of the images, the title of the plot will change to:

"END OF IMAGES. CONGRATULATIONS. STOP LABELLING"

![QAQC](example_images/gui_qaqc_end.png)